# Scikit-Learn Reference

---

# Table of Contents

## [Data Class ](#Data-Class)

## [Load Data ](#Load-Data)

## [Prep Data ](#Prep-Data)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Bag of Words Vectorization ](#Bag-of-Words-Vectorization)

## [Classification ](#Classification)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Linear SVM ](#Linear-SVM)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Decision Tree ](#Decision-Tree)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Naive Bayes ](#Naive-Bayes)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Logistic-Regression ](#Logistic-Regression)

## [Evaluation ](#Evaluation)

---

# Data Class

In [1]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: # score of 4 or 5
            return Sentiment.POSITIVE
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
        
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

---

# Load Data

In [2]:
import json

file_name = "./data/sentiment/books_small_10000.json"
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review["reviewText"], review["overall"]))
reviews[342].sentiment

'POSITIVE'

---

# Prep Data

In [3]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [4]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


### Bag of Words Vectorization

In [5]:
 from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x_vectors[0])

  (0, 4277)	2
  (0, 8608)	1
  (0, 4190)	1
  (0, 8052)	1
  (0, 3068)	1
  (0, 5589)	1
  (0, 3899)	1
  (0, 5354)	1
  (0, 7303)	1
  (0, 5478)	1
  (0, 7929)	3
  (0, 6145)	1
  (0, 4034)	1
  (0, 991)	1
  (0, 525)	1
  (0, 1790)	1
  (0, 3669)	1
  (0, 805)	1
  (0, 5563)	1
  (0, 870)	1
  (0, 1168)	1
  (0, 2688)	1
  (0, 7603)	1
  (0, 4954)	1


---

# Classification

### Linear SVM

In [6]:
from sklearn import svm
clf_svm = svm.SVC(kernel="linear")
clf_svm.fit(train_x_vectors, train_y)
print(clf_svm.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['POSITIVE']
With so much action going on and so many perspectives to tell the story from, &#34;Black Hawk Down&#34; give a truly in-depth look at what happened over the course of less than a day, but just like the fog of war confuses those within it, the mashing of multiple story lines challenges the reader to do lots of work on keeping up with who is where and doing what. This gets easier as the book moves along because the names and locations become more familiar, but to truly understand the first half of the book, one must read it again with the accumulated knowledge of having read the entire book. Other than that, it was written so that the non-military mind can understand the various situations with limited knowledge of tactical jargon and that's a good thing for those of us who do not have a military background but enjoy reading books about the history of our military to get a better appreciation at what they do to provide security for Americans and allies alike. 
 POSITIVE


### Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

print(clf_dec.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['POSITIVE']
With so much action going on and so many perspectives to tell the story from, &#34;Black Hawk Down&#34; give a truly in-depth look at what happened over the course of less than a day, but just like the fog of war confuses those within it, the mashing of multiple story lines challenges the reader to do lots of work on keeping up with who is where and doing what. This gets easier as the book moves along because the names and locations become more familiar, but to truly understand the first half of the book, one must read it again with the accumulated knowledge of having read the entire book. Other than that, it was written so that the non-military mind can understand the various situations with limited knowledge of tactical jargon and that's a good thing for those of us who do not have a military background but enjoy reading books about the history of our military to get a better appreciation at what they do to provide security for Americans and allies alike. 
 POSITIVE


### Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

print(clf_gnb.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['POSITIVE']
With so much action going on and so many perspectives to tell the story from, &#34;Black Hawk Down&#34; give a truly in-depth look at what happened over the course of less than a day, but just like the fog of war confuses those within it, the mashing of multiple story lines challenges the reader to do lots of work on keeping up with who is where and doing what. This gets easier as the book moves along because the names and locations become more familiar, but to truly understand the first half of the book, one must read it again with the accumulated knowledge of having read the entire book. Other than that, it was written so that the non-military mind can understand the various situations with limited knowledge of tactical jargon and that's a good thing for those of us who do not have a military background but enjoy reading books about the history of our military to get a better appreciation at what they do to provide security for Americans and allies alike. 
 POSITIVE


### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

clf_log = DecisionTreeClassifier()
clf_log.fit(train_x_vectors, train_y)

print(clf_log.predict(test_x_vectors[0]))

print(test_x[0], "\n", test_y[0])

['POSITIVE']
With so much action going on and so many perspectives to tell the story from, &#34;Black Hawk Down&#34; give a truly in-depth look at what happened over the course of less than a day, but just like the fog of war confuses those within it, the mashing of multiple story lines challenges the reader to do lots of work on keeping up with who is where and doing what. This gets easier as the book moves along because the names and locations become more familiar, but to truly understand the first half of the book, one must read it again with the accumulated knowledge of having read the entire book. Other than that, it was written so that the non-military mind can understand the various situations with limited knowledge of tactical jargon and that's a good thing for those of us who do not have a military background but enjoy reading books about the history of our military to get a better appreciation at what they do to provide security for Americans and allies alike. 
 POSITIVE


---

# Evaluation

In [10]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.7980769230769231
0.6274038461538461
0.6490384615384616
0.6442307692307693


In [11]:
# F1 Score
from sklearn.metrics import f1_score
print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_gnb.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_log.predict(test_x_vectors), average=None,
         labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))

[0.8028169  0.79310345]
[0.63529412 0.61916462]
[0.6507177 0.647343 ]
[0.63902439 0.6492891 ]


In [12]:
train_y.count(Sentiment.POSITIVE)

436